# HW4

## GENERALIZATION ERROR

### NOTES

- d<sub>vc</sub> = VC dimension, highest number of points shatterable by hypothesis set H (m<sub>H</sub>(N) = 2<sup>N</sup>
- &delta; = 4 * m<sub>H</sub>(2N) e<sup>(-N/8) * &epsilon;<sup>2</sup></sup>
&rArr; ln(&delta;/4) - ln(m<sub>H</sub>(2N)) = (-N/8) * &epsilon;<sup>2</sup> 
&rArr; (8/N) * (ln (m<sub>H</sub>(2N)) - ln(&delta;/4)) = &epsilon;<sup>2</sup>
- For N &gt; d<sub>vc</sub>, use simple approximation m<sub>H</sub>(N) &asymp; N<sup>d<sub>vc</sub></sup>
&rArr; (8/N) * (ln ((2N)<sup>d<sub>vc</sub></sup>) - ln(&delta;/4)) = &epsilon;<sup>2</sup>
&rArr; (8/N) * ((d<sub>vc</sub> * ln(2N)) - ln(&delta;/4)) = &epsilon;<sup>2</sup>